In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as md
import os
import mne
import random
from sklearn.metrics import classification_report

In [3]:
MDD_dataset_Filenames = os.listdir('./data/MDD')
H_dataset_Filenames = os.listdir('./data/H')
MDD_dataset_list = []
H_dataset_list = []

for filename in MDD_dataset_Filenames:
    data = mne.io.read_raw_edf('./dataset/EC/' + str(filename))
    MDD_dataset_list.append(data)  
    
for filename in H_dataset_Filenames:
    data = mne.io.read_raw_edf('./dataset/EC/' + str(filename))
    H_dataset_list.append(data)  



Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S1 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S10 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S11 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S12 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S13 EC.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Extracting EDF parameters from C:\python\study\4월3주\depression\dataset\EC\MDD S14 EC.edf...
EDF file detected
Setting channel 

In [4]:
MDD_dataframe_list = [] #MDD 환자의 데이터프레임 리스트
H_dataframe_list = [] #건강한 사람의 데이터프레임 리스트
for raw in MDD_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    MDD_dataframe_list.append(temp)
    
for raw in H_dataset_list:
    raw_data = raw.get_data(return_times=True)
    channels = raw.ch_names
    T_data = np.transpose(raw_data[0])
    time = np.transpose(raw_data[1])
    temp = pd.DataFrame(T_data, columns = channels, index = time)
    H_dataframe_list.append(temp)

MDD_dataframe_list[23].rename(columns = {'E F3-Ref': 'F3'}, inplace = True)

In [5]:
X = {}
y = []
for i in range(len(MDD_dataframe_list)):
    a = MDD_dataframe_list[i]
    #데이터 클린징
    a.F3 = a.F3[a.F3 > -0.0005]
    a = a.fillna(method='ffill')
    fft = np.fft.fft(a['F3'])
    fft = np.abs(fft)
    X[i] = fft
    y.append(1)
    
for i in range(30, 30+len(H_dataframe_list)):
    a = H_dataframe_list[30-i]
    a.rename(columns = {'EEG F3-LE' : 'F3'}, inplace = True)
    #데이터 클린징
    a.F3 = a.F3[a.F3 > -0.0005]
    a = a.fillna(method='ffill')
    fft = np.fft.fft(a['F3'])
    fft = np.abs(fft)
    X[i] = fft
    y.append(0)
    
X = pd.DataFrame.from_dict(X, orient='index')
X['target'] = y

In [6]:
X = X.fillna(0)

In [7]:
# 타겟 컬럼과 특징 컬럼 정의

target_col = 'target'
target = X[target_col]
features = X[X.columns.difference(['target'])]


In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.5)

In [18]:
from sklearn.svm import SVC
#SVM Model
svm_linear = SVC(kernel='linear', random_state=0)
svm_linear.fit(X_train, y_train)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='linear', max_iter=-1, probability=False, random_state=0,
    shrinking=True, tol=0.001, verbose=False)

In [19]:
#SVM 선형 평가
print("테스트 세트 예측 : ", svm_linear.predict(X_test))
print("테스트 세트 정확도 : ", svm_linear.score(X_test, y_test))

y_pred = svm_linear.predict(X_test)
print(classification_report(y_test, y_pred))


테스트 세트 예측 :  [1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1]
테스트 세트 정확도 :  0.7391304347826086
              precision    recall  f1-score   support

           0       1.00      0.14      0.25         7
           1       0.73      1.00      0.84        16

    accuracy                           0.74        23
   macro avg       0.86      0.57      0.55        23
weighted avg       0.81      0.74      0.66        23



In [20]:
from keras import models
from keras import layers
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [21]:
model = models.Sequential()
model.add(layers.Dense(64, activation = 'relu', input_shape = (77312,)))
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(1, activation = 'sigmoid'))


model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy', recall_m, precision_m, f1_m])

history = model.fit(X_train,
                    y_train,
                    epochs=10,
                    batch_size=512,
                    validation_data=(X_test, y_test))

Train on 22 samples, validate on 23 samples
Epoch 1/10
22/22 [==============================] - 2s 89ms/step - loss: 0.6929 - accuracy: 0.5455 - recall_m: 0.5000 - precision_m: 0.7000 - f1_m: 0.5833 - val_loss: 0.6619 - val_accuracy: 0.7826 - val_recall_m: 0.7500 - val_precision_m: 0.9231 - val_f1_m: 0.8276
Epoch 2/10
22/22 [==============================] - 0s 16ms/step - loss: 0.6444 - accuracy: 0.9545 - recall_m: 0.9286 - precision_m: 1.0000 - f1_m: 0.9630 - val_loss: 0.5938 - val_accuracy: 0.6957 - val_recall_m: 1.0000 - val_precision_m: 0.6957 - val_f1_m: 0.8205
Epoch 3/10
22/22 [==============================] - 0s 12ms/step - loss: 0.5289 - accuracy: 0.8182 - recall_m: 1.0000 - precision_m: 0.7778 - f1_m: 0.8750 - val_loss: 0.5264 - val_accuracy: 0.8696 - val_recall_m: 0.8750 - val_precision_m: 0.9333 - val_f1_m: 0.9032
Epoch 4/10
22/22 [==============================] - 0s 12ms/step - loss: 0.4462 - accuracy: 0.9545 - recall_m: 0.9286 - precision_m: 1.0000 - f1_m: 0.9630 - val_

In [29]:
results = model.evaluate(X_test, y_test)
results

23/23 [==============================] - 0s 5ms/step


list

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 64)                4948032   
_________________________________________________________________
dense_5 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 65        
Total params: 4,952,257
Trainable params: 4,952,257
Non-trainable params: 0
_________________________________________________________________


In [28]:
mse = model.evaluate(X_test, y_test, batch_size=1)
print("mse{}".format(mse))

23/23 [==============================] - 0s 12ms/step
mse[0.31077599630731606, 0.9130434989929199, 0.6956521272659302, 0.6956521272659302, 0.6956521272659302]
